<a href="https://colab.research.google.com/github/TapasKumarDutta1/Multihead_attention/blob/master/one_after_another_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def block(inputs,f):
    a=Conv2D(f,3,padding='same')(inputs)
    a=BatchNormalization()(a)
    a=ReLU()(a)
    b=Conv2D(f,3,padding='same')(a)
    b=BatchNormalization()(b)
    b=ReLU()(b)
    c=Conv2D(f,3,padding='same')(b)
    c=BatchNormalization()(c)
    c=ReLU()(c)
    d=Conv2D(f,3,padding='same')(c)
    d=BatchNormalization()(d)
    d=ReLU()(d)
    mid=Concatenate()([a,b,c,d])
    
    x=Conv2D(f*4,1)(inputs)
    x=BatchNormalization()(x)
    x=ReLU()(x)
    x=Add()([mid,x])
    
    y=Conv2D(f*2,1)(x)
    y=BatchNormalization()(y)
    y=ReLU()(y)
    return y
def Global_attention_block(C_A):
#     shape=K.int_shape(inputs)
#     x=AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
#     x=Conv2D(shape[3]//4,1, padding='same') (x)
#     x=Activation('relu') (x)
#     x=Conv2D(shape[3],1, padding='same') (x)
#     x=Activation('sigmoid') (x)
#     C_A=Multiply()([x,inputs])
    
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (C_A)
    x=Conv2D(1,1, padding='same') (x)
    x=Activation('sigmoid') (x)
    S_A=Multiply()([x,C_A])
    return S_A

def self_attention(inp):
    shp=inp.shape
    a=Conv2D(shp[-1]//8,1, padding='same') (inp)
    a=Activation('sigmoid') (a)
    b=Conv2D(shp[-1]//8,1, padding='same') (inp)
    b=Activation('sigmoid') (b)
    c=Conv2D(shp[-1]//8,1, padding='same') (inp)
    c=Activation('sigmoid') (c)
    
    a=Reshape(( shp[1]*shp[2], shp[3]//8))(a)
    b=Reshape(( shp[1]*shp[2], shp[3]//8))(b)
    b=K.permute_dimensions(b, (0, 2, 1))
    c=Reshape(( shp[1]*shp[2], shp[3]//8))(c)
    inter=K.batch_dot(a,b)
    out=K.batch_dot(inter,c)
    out=Reshape(( shp[1],shp[2], shp[3]//8))(out)
    out=Conv2D(shp[-1],1, padding='same') (out)
    out=Reshape((shp[1],shp[2], shp[3]))(out)
    
    return out
def load_model():   
  K.clear_session() 
  inputs = Input(shape=(224,224,3))
  x=Conv2D(16,3,padding='same')(inputs)
  x=BatchNormalization()(x)
  x=ReLU()(x)
  x=MaxPooling2D()(x)
  
  x=Conv2D(16,3,padding='same')(x)
  x=BatchNormalization()(x)
  x=ReLU()(x)
  x=MaxPooling2D()(x)
    
  a1=block(x,32)
  x=MaxPooling2D()(a1)

  a2=block(x,64)
  x=MaxPooling2D()(a2)
    
  a3=block(x,128)
  a3=self_attention(a3)
  a3=Global_attention_block(a3)
  x=GlobalAveragePooling2D()(a3)
    
  x=Dropout(0.5)(x)
  x=Dense(3,activation='softmax')(x)
  model = Model(inputs=inputs, outputs=x)
  
  return model

